In [ ]:
import logging
import pandas as pd
import datetime
import seaborn as sb
import time
from ipywidgets import interact, fixed
from libs.CovidTimeseriesModel import CovidTimeseriesModel
from libs.CovidDatasets import CovidDatasets

## Constants
start_time = time.time()
HOSPITALIZATION_RATE = .073
HOSPITALIZED_CASES_REQUIRING_ICU_CARE = .1397
TOTAL_INFECTED_PERIOD = 12
MODEL_INTERVAL = 4
r0 = 2.4
INTERVENTIONS = {
        datetime.date(2020, 3, 23): 1.3,
        datetime.date(2020, 4, 20): 1.1,
        datetime.date(2020, 5, 22): 0.8,
        datetime.date(2020, 6, 23): r0
    }
# Pack all of the assumptions and parameters into a dict that can be passed into the model
MODEL_PARAMETERS = {
    # Pack the changeable model parameters
    'timeseries': CovidDatasets().get_timeseries_by_country_state('USA', 'TX'),
    'beds': CovidDatasets().get_beds_by_country_state('USA', 'AK'),
    'population': CovidDatasets().get_population_by_country_state('USA', 'TX'),
    'projection_iterations': 25, # Number of iterations into the future to project
    'r0': r0,
    'interventions': INTERVENTIONS,
    'hospitalization_rate': HOSPITALIZATION_RATE,
    'initial_hospitalization_rate': .05,
    'case_fatality_rate': .011,
    'hospitalized_cases_requiring_icu_care': HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
    # Assumes that anyone who needs ICU care and doesn't get it dies
    'case_fatality_rate_hospitals_overwhelmed': HOSPITALIZATION_RATE * HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
    'hospital_capacity_change_daily_rate': 1.05,
    'max_hospital_capacity_factor': 2.07,
    'initial_hospital_bed_utilization': .6,
    'model_interval': 4, # In days
    'total_infected_period': 12, # In days
    'rolling_intervals_for_current_infected': int(round(TOTAL_INFECTED_PERIOD / MODEL_INTERVAL, 0)),
}

def run_model(model_parameters):
    return CovidTimeseriesModel().forecast_region(model_parameters)

ak = run_model(model_parameters=MODEL_PARAMETERS)

sb.lineplot(x='Timestamp', y='Cum. Deaths', data=ak)